This code is generated by an AI. works fine, does my intended job, it rectifies all the mistakes i did, typos, errors, and all. 

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

BASE_URL = "https://www.bbc.com"
session = requests.Session()

response = session.get(BASE_URL)
soup = BeautifulSoup(response.text, "html.parser")

articles = []

cards = soup.select('div[data-testid="dundee-article"]')

for card in cards:
    a_tag = card.find_parent("a")
    if not a_tag:
        continue

    url = urljoin(BASE_URL, a_tag["href"])

    headline = card.select_one('[data-testid="card-headline"]')
    short_desc = card.select_one('[data-testid="card-description"]')

    articles.append({
        "url": url,
        "title": headline.get_text(strip=True) if headline else None,
        "short_desc": short_desc.get_text(strip=True) if short_desc else None
    })

print(f"Collected {len(articles)} article links")


Collected 21 article links


In [2]:
def scrape_article(url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Reporter / author
    reporter_tag = soup.select_one('span[data-testid="byline-name"]')
    reporter = reporter_tag.get_text(strip=True) if reporter_tag else None

    # Full article text
    paragraphs = soup.select('div[data-component="text-block"] p')
    full_text = " ".join(p.get_text(strip=True) for p in paragraphs)

    return reporter, full_text


In [10]:
final_data = []

for article in articles:   # limit while testing
    reporter, full_text = scrape_article(article["url"])

    final_data.append({
        "title": article["title"],
        "short_desc": article["short_desc"],
        "reporter": reporter,
        "content": full_text,
        "url": article["url"]
    })

    time.sleep(1)


In [11]:
final_data[0].keys()

dict_keys(['title', 'short_desc', 'reporter', 'content', 'url'])

In [12]:
import pandas as pd

df = pd.DataFrame(final_data)
df.to_csv("bbc_news_dataset.csv",index=False)